# MONAI : Tutorial : modules
## 3d_image_transforms

参照URL:
- https://github.com/Project-MONAI/tutorials/blob/main/modules/3d_image_transforms.ipynb

## 0. 準備

In [1]:
# パッケージのインストール
#!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
#!python -c "import matplotlib" || pip install -q matplotlib

In [ ]:
# パッケージのインポート
import warnings
warnings.simplefilter('ignore')

import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

from monai.apps import download_and_extract
from monai.config import print_config
from monai.transforms import (
    EnsureChannelFirstd,
    LoadImage,
    LoadImaged,
    Orientationd,
    Rand3DElasticd,
    RandAffined,
    Spacingd,
)

print_config()

In [ ]:
# データフォルダ
root_dir = './data/MSD'

In [ ]:
# データダウンロード
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar"
md5 = "410d4a301da4e5b2f6f86ec3ddba524e"

compressed_file = os.path.join(root_dir, 'Task09_Spleen.tar')
data_dir = os.path.join(root_dir, 'Task09_Spleen')
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

## 1. データセット

In [ ]:
train_images = sorted(glob(os.path.join(data_dir, 'imagesTr', '*.nii.gz')))
train_labels = sorted(glob(os.path.join(data_dir, 'labelsTr', '*.nii.gz')))

data_dicts = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(train_images, train_labels)]
train_data_dicts, val_data_dicts = data_dicts[:-9], data_dicts[-9:]

In [ ]:
train_data_dicts[0]

### 1.1 NlfTlファルのロード

In [ ]:
loader = LoadImage(dtype=np.float32, image_only=True)

In [ ]:
image = loader(train_data_dicts[0]['image'])

print(f"image shape: {image.shape}")
print(f"image affine:\n{image.meta['affine']}")
print(f"image pixdim:\n{image.pixdim}")

In [ ]:
loader = LoadImaged(keys=['image', 'label'], image_only=False)

In [ ]:
data_dict = loader(train_data_dicts[0])

print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['label'].shape}")
print(f"image pixdim:\n{data_dict['image'].pixdim}")

In [ ]:
image, label = data_dict['image'], data_dict['label']
plt.figure('visualize', figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[:, :, 30], cmap='gray')

plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[:, :, 30])
plt.show()

### 1.2 1次元の確認

In [ ]:
ensure_channel_first = EnsureChannelFirstd(keys=['image', 'label'])
datac_dict = ensure_channel_first(data_dict)
print(f"image shaep: {datac_dict['image'].shape}")

### 1.3 軸コードの方向転換

In [ ]:
orientation = Orientationd(keys=['image', 'label'], axcodes='PLI')

In [ ]:
data_dict = orientation(datac_dict)

print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['label'].shape}")
print(f"image affine after spacing:\n{data_dict['image'].meta['affine']}")
print(f"label affine after spacing:\n{data_dict['label'].meta['affine']}")

In [ ]:
image, label = data_dict['image'], data_dict['label']
plt.figure('visualize', figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[0, :, :, 30], cmap='gray')
plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[0, :, :, 30])
plt.show()

### 1.4 ボクセルサイズへのリサンプリング

In [ ]:
spacing = Spacingd(keys=['image', 'label'], pixdim=(1.5, 1.5, 5.0), mode=('bilinear', 'nearest'))

In [ ]:
data_dict = spacing(data_dict)

print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['label'].shape}")
print(f"image affine after Spacing:\n{data_dict['image'].meta['affine']}")
print(f"label affine after Spacing:\n{data_dict['label'].meta['affine']}")

In [ ]:
image, label = data_dict['image'], data_dict['label']

plt.figure('visualize', figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[0, :, :, 30], cmap='gray')

plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[0, :, :, 30])
plt.show()

### 1.5 ランダムなアフィン変換

In [ ]:
rand_affine = RandAffined(
    keys=['image', 'label'],
    mode=('bilinear', 'nearest'),
    prob=1.0,
    spatial_size=(300, 300, 50),
    translate_range=(40, 40, 2),
    rotate_range=(np.pi / 36, np.pi / 36, np.pi / 4),
    scale_range=(0.15, 0.15, 0.15),
    padding_mode='border',
)

rand_affine.set_random_state(seed=123)

In [ ]:
affined_data_dict = rand_affine(data_dict)
print(f"image shape: {affined_data_dict['image'].shape}")


In [ ]:
image, label = affined_data_dict['image'][0], affined_data_dict['label'][0]

plt.figure('visualize', figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[:, :, 23], cmap='gray')

plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[:, :, 23])
plt.show()

### 1.6 ランダムな弾性変形

In [ ]:
rand_elastic = Rand3DElasticd(
    keys=['image', 'label'],
    mode=('bilinear', 'nearest'),
    prob=1.0,
    sigma_range=(5, 8),
    magnitude_range=(100, 200),
    spatial_size=(300, 300, 10),
    translate_range=(50, 50, 2),
    rotate_range=(np.pi / 36, np.pi / 36, np.pi),
    scale_range=(0.15, 0.15, 0.15),
    padding_mode='border',
)

rand_elastic.set_random_state(seed=123)

In [ ]:
deformed_data_dict = rand_elastic(data_dict)
print(f"image shape: {deformed_data_dict['image'].shape}")

In [ ]:
image, label = deformed_data_dict['image'][0], deformed_data_dict['label'][0]

plt.figure('visualize')
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[:, :, 5], cmap='gray')

plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[:, :, 5])
plt.show()
